<a href="https://colab.research.google.com/github/SARITHASAKTHIVEL/Security-dashboard/blob/main/Bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
import torch

In [6]:
data = pd.read_csv('/content/rule-engine.csv')

In [7]:
print(data.head())       # Display the first few rows
print(data.columns)

    Category                      Field type of data  Label Unnamed: 4  \
0  financial      [a-zA-Z0-9._%+-]+@apl       UPI ID     10        NaN   
1  financial     [a-zA-Z0-9._%+-]+@yapl       UPI ID     10        NaN   
2  financial     [a-zA-Z0-9._%+-]+@rapl       UPI ID     10        NaN   
3  financial  [a-zA-Z0-9._%+-]+@abfspay       UPI ID     10        NaN   
4  financial  [a-zA-Z0-9._%+-]+@abfspay       UPI ID     10        NaN   

   Unnamed: 5  
0         NaN  
1         NaN  
2         NaN  
3         NaN  
4         NaN  
Index(['Category', 'Field', 'type of data', 'Label', 'Unnamed: 4',
       'Unnamed: 5'],
      dtype='object')


In [8]:
data.columns = data.columns.str.strip()

In [9]:
data['text'] = data['Field'].astype(str)

In [10]:
label_encoder = LabelEncoder()
data['Label'] = label_encoder.fit_transform(data['Label'])

In [11]:
train_texts, test_texts, train_labels, test_labels, train_exposure, test_exposure = train_test_split(
    data['text'].tolist(), data['Label'].tolist(), data['Label'].tolist(), test_size=0.2, random_state=42
)

In [12]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [13]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=128)


In [14]:
class SensitiveDataDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels, exposure):
        self.encodings = encodings
        self.labels = labels
        self.exposure = exposure

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])  # Now this should be an integer
        item['exposure_rate'] = torch.tensor(self.exposure[idx])  # Add exposure rate if needed
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = SensitiveDataDataset(train_encodings, train_labels, train_exposure)
test_dataset = SensitiveDataDataset(test_encodings, test_labels, test_exposure)

In [17]:
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=len(label_encoder.classes_))

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_dir='./logs',
    evaluation_strategy="epoch",  # Evaluate at the end of each epoch
    save_total_limit=2,  # Save only the last 2 models
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [18]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)


In [19]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,0.374195
2,No log,0.246387
3,No log,0.190236


TrainOutput(global_step=156, training_loss=0.40318503746619594, metrics={'train_runtime': 703.4555, 'train_samples_per_second': 1.774, 'train_steps_per_second': 0.222, 'total_flos': 29385020875392.0, 'train_loss': 0.40318503746619594, 'epoch': 3.0})

In [21]:
from sklearn.metrics import accuracy_score

# Get predictions from the model
predictions = trainer.predict(test_dataset)

# Extract predicted labels and true labels
predicted_labels = predictions.predictions.argmax(axis=1)  # Get the predicted class
true_labels = predictions.label_ids

# Calculate accuracy
accuracy = accuracy_score(true_labels, predicted_labels)

print(f"Accuracy: {accuracy * 100:.2f}%")


Accuracy: 94.23%
